In [1]:
import os
import logging
from datetime import datetime
from typing import TypedDict, Dict, Any, Optional
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import time
import re
import subprocess
import shutil
import json
from pathlib import Path
from functools import wraps


In [2]:
# 1. Environment Setup & Configuration
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found in .env file. Please set it.")

# Output directories
OUTPUT_FOLDER = "input_data_output_16"
LOGS_FOLDER_NAME = "logs"
REPORTS_FOLDER_NAME = "reports"

# Create main output directory
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Setup logging
log_filename = os.path.join(OUTPUT_FOLDER, LOGS_FOLDER_NAME,
                            f"universal_converter_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
os.makedirs(os.path.join(OUTPUT_FOLDER, LOGS_FOLDER_NAME), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] - %(message)s',
    handlers=[
        logging.FileHandler(log_filename, encoding="utf-8"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


In [3]:
# 2. State Definition
class AgentState(TypedDict):
    user_story: str
    code_input: str
    gherkin_feature: str
    analyzed_code: str
    test_plan: str
    playwright_code: str
    review_feedback: str
    final_playwright_code: str
    code_path: str
    test_results: Optional[str]
    test_passed: Optional[bool]
    coverage_report: Optional[str]
    coverage_percentage: Optional[float]
    output_dir: str
    artifacts: Dict[str, str]


In [4]:
# 3. Initialize LLM
def get_llm(temperature=0.1):
    """Initializes and returns the ChatGroq LLM."""
    return ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama3-70b-8192",
        temperature=temperature,
        max_tokens=4096,
        request_timeout=60
    )


In [5]:
# 4. Enhanced Retry Decorator
def with_retry(func=None, *, max_attempts=3, base_delay=3):
    def decorator(inner_func):
        @wraps(inner_func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    return inner_func(*args, **kwargs)
                except Exception as e:
                    delay = base_delay * (2 ** (attempt - 1))
                    if "429" in str(e) or "Too Many Requests" in str(e):
                        logger.warning(f"Rate limit hit in {inner_func.__name__}, waiting {delay}s before retry {attempt}/{max_attempts}")
                    else:
                        logger.warning(f"Attempt {attempt} failed in {inner_func.__name__}: {e}")
                    if attempt == max_attempts:
                        logger.error(f"Max retries reached for {inner_func.__name__}. Error: {e}")
                        raise
                    time.sleep(delay)
        return wrapper
    return decorator(func) if func else decorator


In [6]:
# 5. Helper Functions
def save_artifact(artifact_dir: str, filename: str, content: str) -> str:
    """Save content to a file in the specified directory."""
    if not content or not content.strip():
        logger.warning(f"Skipping empty artifact: {filename}")
        return ""
    
    os.makedirs(artifact_dir, exist_ok=True)
    file_path = os.path.join(artifact_dir, filename)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)
    logger.info(f"Saved artifact: {file_path}")
    return file_path

def save_json_artifact(artifact_dir: str, filename: str, data: dict) -> str:
    """Save JSON data to a file in the specified directory."""
    if not data:
        logger.warning(f"Skipping empty JSON artifact: {filename}")
        return ""
    
    os.makedirs(artifact_dir, exist_ok=True)
    file_path = os.path.join(artifact_dir, filename)
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    logger.info(f"Saved JSON artifact: {file_path}")
    return file_path

def load_test_cases(input_folder: str = "input_data_1") -> list:
    """Load test cases from the input folder."""
    test_cases = []
    if not os.path.exists(input_folder):
        logger.error(f"Input folder '{input_folder}' not found.")
        return []
    logger.info(f"Loading test cases from: {input_folder}")
    for item in os.listdir(input_folder):
        item_path = os.path.join(input_folder, item)
        if not os.path.isdir(item_path):
            continue
        try:
            user_story_path = os.path.join(item_path, "user_story.txt")
            user_story = ""
            if os.path.exists(user_story_path):
                with open(user_story_path, "r", encoding="utf-8") as f:
                    user_story = f.read().strip()
            code_input, code_path = "", ""
            code_extensions = (".js", ".jsx", ".ts", ".tsx", ".cy.js", ".spec.js", ".test.js")
            for file_name in os.listdir(item_path):
                if file_name.endswith(code_extensions):
                    code_path = os.path.join(item_path, file_name)
                    with open(code_path, "r", encoding="utf-8") as f:
                        code_input = f.read().strip()
                    break
            if code_input:
                test_cases.append({
                    "name": item,
                    "user_story": user_story,
                    "code_input": code_input,
                    "code_path": code_path
                })
                logger.info(f"Loaded: {item}")
            else:
                logger.warning(f"Incomplete: {item}")
        except Exception as e:
            logger.error(f"Error loading {item}: {e}")
    logger.info(f" Total loaded: {len(test_cases)} test cases")
    return test_cases



In [7]:
# 6. Agent Definitions
@with_retry
def synthetic_user_story_generator(state: AgentState) -> dict:
    """Generate synthetic user stories from source code if user_story is missing."""
    logger.info(" Agent: SyntheticUserStoryGenerator - Creating user story from code.")
    
    if state.get("user_story", "").strip():
        logger.info("User story already exists. Skipping synthetic generation.")
        return {}
    if not state.get("code_input", "").strip():
        logger.warning(" Cannot generate user story: no code input available.")
        synthetic_story = "As a user, I want the app to work, so that I can complete tasks."
    else:
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an AI trained to write user stories from raw frontend code. 
            Generate a realistic user story describing what the code does, as if it were written by a product owner.
            Format: "As a [role], I want to [goal], so that [benefit]."
            Return ONLY the user story as plain text."""),
            ("user", "Here is the source code:\n{code_input}")
        ])
        chain = prompt | get_llm(0.7) | StrOutputParser()
        synthetic_story = chain.invoke({"code_input": state["code_input"]}).strip()
        time.sleep(1)
    
    return {
        "user_story": synthetic_story,
    }



In [8]:
@with_retry
def gherkin_converter(state: AgentState) -> dict:
    """Convert a user story into a Gherkin .feature file format."""
    logger.info("Agent: GherkinConverter - Converting user story to Gherkin BDD format.")
    if not state.get("user_story", "").strip():
        return {"gherkin_feature": "# Feature: No user story provided\n# Scenario: Empty input"}
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Convert the given user story into a well-structured Gherkin .feature file.
        Requirements:
        - Create descriptive Feature and Scenario names
        - Use proper Given/When/Then structure
        - Use 2-space indentation
        - Return only the raw Gherkin content without markdown formatting"""),
        ("user", "Convert this user story to Gherkin:\n{user_story}")
    ])
    chain = prompt | get_llm(0.7) | StrOutputParser()
    feature = chain.invoke({"user_story": state["user_story"]})
    feature = feature.strip()
    
    return {
        "gherkin_feature": feature,
    }


In [9]:
@with_retry
def code_analyzer(state: AgentState) -> dict:
    """Analyze frontend code to extract key information for testing."""
    logger.info("Agent: CodeAnalyzer - Analyzing input code.")
    if not state.get("code_input", "").strip():
        return {"analyzed_code": "No code input was provided for analysis."}
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Analyze the provided code and extract key testing elements.
        Focus on:
        1. Purpose and main functionality
        2. User interactions (clicks, form submissions, navigation)
        3. Selectors (CSS, IDs, class names used)
        4. Assertions or test conditions
        5. API calls and navigation URLs
        6. Form fields and input elements
        7. Test data values used in the original code
        Provide a concise, structured analysis in plain text. Preserve all original test data values exactly as they appear in the code."""),
        ("user", "Analyze this code for testing:\n{code_input}")
    ])
    chain = prompt | get_llm(0.3) | StrOutputParser()
    analysis = chain.invoke({"code_input": state["code_input"]})
    analysis = analysis.strip()
    time.sleep(2)
    
    return {
        "analyzed_code": analysis,
    }

In [10]:
@with_retry
def test_plan_generator(state: AgentState) -> dict:
    """Create a detailed test plan for Playwright."""
    logger.info("Agent: TestPlanGenerator - Creating comprehensive test plan.")
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Create a detailed Playwright test plan based on the Gherkin feature and code analysis.
        Write it as a simple text description without code blocks or markdown.
        CRITICAL REQUIREMENTS:
        1. Follow the original code structure exactly - do not add describe blocks, beforeEach blocks, or any other structure not present in the original code
        2. Use the exact same test data values as in the original code
        3. Include only the actions and assertions that are present in the original code
        4. Do not add any additional steps, assertions, or actions
        5. Map Cypress commands directly to their Playwright equivalents:
           - cy.visit() → page.goto()
           - cy.url().should() → expect(page).toHaveURL()
           - cy.get().type() → page.locator().fill()
           - cy.get().should('have.value') → expect(page.locator()).toHaveValue()
           - cy.get().click() → page.locator().click()
        6. Preserve the order of operations exactly as in the original code
        7. DO NOT add any waitFor, waitForLoadState, or other wait methods that weren't in the original code
        Include:
        1. Navigation (page URLs and routing)
        2. Locators (specific selectors to use, exactly as in the original)
        3. Actions (step-by-step user interactions, exactly as in the original)
        4. Assertions (what to verify at each step, exactly as in the original)
        Write as actionable instructions, not code."""),
        ("user", "Create test plan based on:\nGherkin Feature:\n{gherkin_feature}\nCode Analysis:\n{analyzed_code}")
    ])
    chain = prompt | get_llm(0.5) | StrOutputParser()
    plan = chain.invoke({
        "gherkin_feature": state["gherkin_feature"],
        "analyzed_code": state["analyzed_code"]
    }).strip()
    time.sleep(2)
    
    return {
        "test_plan": plan,
    }

In [11]:
@with_retry
def playwright_code_generator(state: AgentState) -> dict:
    """Generate executable Playwright code from the test plan."""
    logger.info("Agent: PlaywrightCodeGenerator - Generating executable test code.")
    test_plan = state.get("test_plan", "").strip()
    if not test_plan:
        logger.error("No test plan provided")
        return {"playwright_code": "// Error: No test plan provided"}
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a Playwright expert. Generate a complete, executable test file (.spec.js) based on the test plan.
        CRITICAL REQUIREMENTS:
        - Start with: const {{ test, expect }} = require('@playwright/test');
        - Use a simple test() structure - NO test.describe() blocks unless explicitly mentioned in the plan
        - Use proper async/await syntax
        - Include the exact test name from the original code
        - Use robust Playwright locators (page.locator, page.getByRole, etc.)
        - Add proper assertions with expect()
        - Use the EXACT same test data values as in the original code
        - Include ONLY the actions and assertions that were in the original code
        - DO NOT add any additional steps, assertions, or actions
        - DO NOT add any comments that weren't in the original code
        - DO NOT add any explanatory text before or after the code
        - DO NOT add any waitFor, waitForLoadState, or other wait methods that weren't in the original code
        - Return ONLY the raw JavaScript code without any formatting or explanations
        - The output should be exactly the code that can be saved as a .spec.js file and executed
        Example structure:
        const {{ test, expect }} = require('@playwright/test');
        test('test description from original', async ({{ page }}) => {{
          await page.goto('url');
          await page.locator('selector').fill('exact value from original');
          await expect(page.locator('selector')).toHaveValue('exact value from original');
        }});"""),
        ("user", "Generate Playwright code for this test plan:\n{test_plan}")
    ])
    chain = prompt | get_llm(0.3) | StrOutputParser()
    code = chain.invoke({"test_plan": test_plan}).strip()
    
    # Clean output
    code = re.sub(r'^.*?(?=const \{ test, expect \} = require)', '', code, flags=re.DOTALL)
    code = re.sub(r'^```(?:javascript)?', '', code, flags=re.MULTILINE)
    code = re.sub(r'^```$', '', code, flags=re.MULTILINE)
    code = code.strip()
    
    import_statement = "const { test, expect } = require('@playwright/test');"
    if not code.startswith("const { test, expect }"):
        code = f"{import_statement}\n{code}"
    
    # Remove unwanted waits
    code = re.sub(r'\s*await page\.waitForLoadState\(.*?\);', '', code)
    code = re.sub(r'\s*await page\.locator\(.*?\)\.waitFor\(.*?\);', '', code)
    code = re.sub(r'\s*await page\.waitForTimeout\(.*?\);', '', code)
    
    lines = code.split('\n')
    cleaned_lines = []
    for line in lines:
        if any(line.strip().startswith(prefix) for prefix in [
            "const { test, expect }", "test(", "async ({ page })", "})", "await page.", "await expect(page.", "});"
        ]):
            cleaned_lines.append(line)
        elif line.strip().startswith("//"):
            continue
        elif line.strip():
            cleaned_lines.append(line)
    
    code = '\n'.join(cleaned_lines)
    time.sleep(2)
    
    return {
        "playwright_code": code,
    }

In [12]:
@with_retry
def code_reviewer(state: AgentState) -> dict:
    """Review the generated Playwright code for improvements."""
    logger.info("Agent: CodeReviewer - Reviewing generated code quality.")
    playwright_code = state.get("playwright_code", "")
    if not playwright_code or playwright_code.startswith("// Error:"):
        return {"review_feedback": "No feedback required."}
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Review this Playwright test code for quality and accuracy compared to the original code.
        Check for:
        1. Syntax correctness
        2. Playwright best practices
        3. Locator reliability
        4. Assertion completeness
        5. Whether the code accurately reflects the original test structure
        6. Whether all original test data values are preserved exactly
        7. Whether any unnecessary elements (describe blocks, beforeEach, etc.) have been added
        8. Whether any extra comments or explanatory text has been added
        If the code is good, executable, and accurately reflects the original, respond EXACTLY with: "No feedback required."
        Otherwise, provide specific improvement suggestions."""),
        ("user", "Review this Playwright code:\n{playwright_code}")
    ])
    chain = prompt | get_llm(0.2) | StrOutputParser()
    feedback = chain.invoke({"playwright_code": playwright_code}).strip()
    time.sleep(2)
    
    return {
        "review_feedback": feedback,
    }

In [13]:
@with_retry
def code_refiner(state: AgentState) -> dict:
    """Refine the Playwright code based on review feedback."""
    logger.info("🔧 Agent: CodeRefiner - Applying improvements to code.")
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Refine the Playwright code based on the provided feedback.
        Requirements:
        - Apply all suggested improvements
        - Maintain original functionality
        - Preserve all original test data values exactly
        - Do not add any elements not present in the original code
        - Do not add any comments that weren't in the original code
        - Do not add any explanatory text before or after the code
        - DO NOT add any waitFor, waitForLoadState, or other wait methods that weren't in the original code
        - Return ONLY the raw JavaScript code without any formatting or explanations
        - The output should be exactly the code that can be saved as a .spec.js file and executed
        Example:
        const {{ test, expect }} = require('@playwright/test');
        test('example', async ({{ page }}) => {{
          await page.goto('https://example.com');
        }});"""),
        ("user", "Improve this code:\nOriginal Code:\n{playwright_code}\nFeedback:\n{review_feedback}")
    ])
    chain = prompt | get_llm(0.3) | StrOutputParser()
    refined_code = chain.invoke({
        "playwright_code": state["playwright_code"],
        "review_feedback": state["review_feedback"]
    }).strip()
    
    # Clean refined code
    refined_code = re.sub(r'^.*?(?=const \{ test, expect \} = require)', '', refined_code, flags=re.DOTALL)
    refined_code = re.sub(r'^```(?:\w+)?', '', refined_code, flags=re.MULTILINE)
    refined_code = re.sub(r'^```$', '', refined_code, flags=re.MULTILINE)
    
    import_statement = "const { test, expect } = require('@playwright/test');"
    if not refined_code.startswith("const { test, expect }"):
        refined_code = f"{import_statement}\n{refined_code}"
    
    # Remove waits
    refined_code = re.sub(r'\s*await page\.waitForLoadState\(.*?\);', '', refined_code)
    refined_code = re.sub(r'\s*await page\.locator\(.*?\)\.waitFor\(.*?\);', '', refined_code)
    refined_code = re.sub(r'\s*await page\.waitForTimeout\(.*?\);', '', refined_code)
    
    lines = refined_code.split('\n')
    cleaned_lines = []
    for line in lines:
        if any(line.strip().startswith(prefix) for prefix in [
            "const { test, expect }", "test(", "async ({ page })", "})", "await page.", "await expect(page.", "});"
        ]):
            cleaned_lines.append(line)
        elif line.strip().startswith("//"):
            continue
        elif line.strip():
            cleaned_lines.append(line)
    
    refined_code = '\n'.join(cleaned_lines)
    time.sleep(2)
    
    return {
        "final_playwright_code": refined_code,
    }



In [14]:
def set_final_code_if_needed(state: AgentState) -> dict:
    """Ensures final_playwright_code is set when no refinement is needed."""
    feedback = state.get("review_feedback", "").lower().strip()
    if "no feedback required" in feedback:
        return {
            "final_playwright_code": state.get("playwright_code", ""),
        }
    return {}

def writer_agent(state: AgentState) -> dict:
    """Organizes artifacts into project structure folders."""
    logger.info("Agent: WriterAgent - Organizing test files and artifacts.")
    
    # Create organized directory structure
    output_dir = state["output_dir"]
    features_dir = os.path.join(output_dir, "features")
    tests_dir = os.path.join(output_dir, "tests")
    reports_dir = os.path.join(output_dir, "reports")
    
    os.makedirs(features_dir, exist_ok=True)
    os.makedirs(tests_dir, exist_ok=True)
    os.makedirs(reports_dir, exist_ok=True)
    
    artifacts = {}
    
    # Save Gherkin feature file
    gherkin_feature = state.get("gherkin_feature", "")
    if gherkin_feature and gherkin_feature.strip():
        feature_path = save_artifact(features_dir, "generated.feature", gherkin_feature)
        if feature_path:
            artifacts["gherkin_feature"] = feature_path
    
    # Save Playwright test code
    final_code = state.get("final_playwright_code", "")
    if final_code and final_code.strip():
        test_path = save_artifact(tests_dir, "generated.spec.js", final_code)
        if test_path:
            artifacts["playwright_code"] = test_path
    
    # Save test execution results if available
    test_results = state.get("test_results")
    if test_results and test_results.strip():
        results_path = save_artifact(reports_dir, "test_results.txt", test_results)
        if results_path:
            artifacts["test_results"] = results_path
    
    # Save JSON test results if available
    test_passed = state.get("test_passed")
    if test_passed is not None:
        json_results = {
            "test_passed": test_passed,
            "timestamp": datetime.now().isoformat()
        }
        json_path = save_json_artifact(reports_dir, "test_results.json", json_results)
        if json_path:
            artifacts["test_results_json"] = json_path
    
    # Save coverage report if available
    coverage_report = state.get("coverage_report")
    if coverage_report and coverage_report.strip():
        artifacts["coverage_report"] = coverage_report
    
    logger.info(f" Organized {len(artifacts)} artifacts into structured folders.")
    return {"artifacts": artifacts}


In [15]:
def playwright_test_runner(state: AgentState) -> dict:
    """Execute the final Playwright test script and report results with coverage."""
    logger.info("Agent: PlaywrightTestRunner - Executing test script with coverage.")
    final_code = state.get("final_playwright_code", "").strip()
    if not final_code:
        logger.error("No final code to execute")
        return {"test_results": "Error: No valid code to execute.", "test_passed": False}
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S_%f')
    temp_test_file = f"temp_test_{timestamp}.spec.js"
    temp_test_path = os.path.join(state["output_dir"], "tests", temp_test_file)
    
    try:
        with open(temp_test_path, "w", encoding="utf-8") as f:
            f.write(final_code)
        logger.info(f"Test file created: {temp_test_path}")
        
        npx_path = shutil.which("npx")
        if not npx_path:
            potential_paths = [
                r"C:\Program Files\nodejs\npx.cmd",
                r"C:\Program Files\nodejs\npx",
                os.path.join(os.environ.get("APPDATA", ""), "npm", "npx.cmd"),
                os.path.join(os.environ.get("PROGRAMFILES", ""), "nodejs", "npx.cmd"),
            ]
            for path in potential_paths:
                if os.path.exists(path):
                    npx_path = path
                    break
        
        if not npx_path:
            logger.error("Could not find npx executable")
            return {
                "test_results": "Error: Could not find npx executable. Please ensure Node.js is properly installed.",
                "test_passed": False
            }
        
        # Check if nyc is available
        nyc_path = shutil.which("nyc")
        use_nyc = nyc_path is not None
        
        coverage_dir = os.path.join(state["output_dir"], "reports", "coverage")
        os.makedirs(coverage_dir, exist_ok=True)
        coverage_percentage = 0.0
        coverage_report_path = ""
        
        env = os.environ.copy()
        path_separator = ";" if os.name == "nt" else ":"
        env["PATH"] = f"{os.path.dirname(npx_path)}{path_separator}{env.get('PATH', '')}"
        env["DEBUG"] = "pw:api"
        
        if use_nyc:
            logger.info("Using nyc for coverage reporting...")
            # Create a temporary package.json for nyc
            package_json_path = os.path.join(state["output_dir"], "package.json")
            package_json = {
                "name": "test-coverage",
                "version": "1.0.0",
                "scripts": {
                    "test": "nyc --reporter=text --reporter=html --report-dir=./reports/coverage npx playwright test " + temp_test_path
                },
                "devDependencies": {
                    "@playwright/test": "^1.30.0",
                    "nyc": "^15.1.0"
                }
            }
            
            with open(package_json_path, "w") as f:
                json.dump(package_json, f, indent=2)
            
            # Create a .nycrc configuration file
            nycrc_path = os.path.join(state["output_dir"], ".nycrc")
            nycrc_config = {
                "reporter": ["text", "html"],
                "report-dir": "./reports/coverage",
                "include": ["**/*.js"],
                "exclude": ["**/node_modules/**", "**/tests/**", "**/coverage/**"],
                "all": True,
                "check-coverage": False,
                "lines": 80,
                "statements": 80,
                "functions": 80,
                "branches": 80
            }
            
            with open(nycrc_path, "w") as f:
                json.dump(nycrc_config, f, indent=2)
            
            # Run nyc to collect coverage
            nyc_command = [nyc_path, "--reporter=text", "--reporter=html", "--report-dir=./reports/coverage", "npx", "playwright", "test", temp_test_path]
            result = subprocess.run(nyc_command, capture_output=True, text=True, timeout=120, env=env, cwd=state["output_dir"])
            
            # Extract coverage percentage from nyc output
            coverage_match = re.search(r'All files\s*\|\s*(\d+\.\d+)', result.stdout)
            if coverage_match:
                coverage_percentage = float(coverage_match.group(1))
                logger.info(f"Coverage: {coverage_percentage:.1f}%")
            
            # Save coverage report path
            coverage_report_path = os.path.join(coverage_dir, "index.html")
            if os.path.exists(coverage_report_path):
                logger.info(f"Coverage report saved to: {coverage_report_path}")
            
            test_passed = result.returncode == 0
            status = "PASSED" if test_passed else "FAILED"
            logger.info(f"Test execution completed: {status}")
            
            # The output from nyc includes both test output and coverage
            execution_summary = f"""
Test Execution Summary (with coverage):
  Status: {status}
  Test File: {temp_test_path}
  Return Code: {result.returncode}
  Coverage: {coverage_percentage:.1f}%
  
Output:
{result.stdout}

Errors:
{result.stderr}
"""
        else:
            logger.info(" Running tests without coverage (nyc not available)...")
            # Run tests without coverage
            command = [npx_path, "playwright", "test", temp_test_path, "--reporter=list", "--timeout=30000", "--retries=0"]
            result = subprocess.run(command, capture_output=True, text=True, timeout=120, env=env)
            
            test_passed = result.returncode == 0
            status = "PASSED" if test_passed else "FAILED"
            logger.info(f"Test execution completed: {status}")
            
            # Create a simple coverage summary
            coverage_summary = f"""
Coverage Summary:
  Test Status: {status}
  Test File: {temp_test_path}
  Coverage Report: Not available (nyc/Istanbul not installed)
  
Test Output:
{result.stdout}

Test Errors:
{result.stderr}
"""
            coverage_report_path = save_artifact(coverage_dir, "coverage_summary.txt", coverage_summary)
            
            execution_summary = f"""
Test Execution Summary:
  Status: {status}
  Test File: {temp_test_path}
  Return Code: {result.returncode}
  
Output:
{result.stdout}

Errors:
{result.stderr}
"""
        
        return {
            "test_results": execution_summary,
            "test_passed": test_passed,
            "coverage_report": coverage_report_path,
            "coverage_percentage": coverage_percentage,
        }
    except subprocess.TimeoutExpired:
        error_msg = "Test execution timed out after 2 minutes"
        logger.error(error_msg)
        return {
            "test_results": f"Error: {error_msg}",
            "test_passed": False,
        }
    except Exception as e:
        error_msg = f"Test execution failed: {str(e)}"
        logger.error(error_msg)
        return {
            "test_results": f"Error: {error_msg}",
            "test_passed": False,
        }
    finally:
        if os.path.exists(temp_test_path):
            try:
                os.remove(temp_test_path)
                logger.info(f"Cleaned up: {temp_test_path}")
            except Exception as e:
                logger.warning(f"Cleanup failed: {e}")


In [16]:
# 7. Build Graph
def decide_to_refine(state: AgentState) -> str:
    feedback = state.get("review_feedback", "").lower().strip()
    if "no feedback required" in feedback:
        logger.info("No refinement needed")
        return "set_final_code"
    else:
        logger.info("Code needs refinement")
        return "refine_code"

def build_graph():
    workflow = StateGraph(AgentState)
    workflow.add_node("synthetic_user_story_generator", synthetic_user_story_generator)
    workflow.add_node("gherkin_converter", gherkin_converter)
    workflow.add_node("code_analyzer", code_analyzer)
    workflow.add_node("test_plan_generator", test_plan_generator)
    workflow.add_node("playwright_code_generator", playwright_code_generator)
    workflow.add_node("code_reviewer", code_reviewer)
    workflow.add_node("code_refiner", code_refiner)
    workflow.add_node("set_final_code", set_final_code_if_needed)
    workflow.add_node("writer_agent", writer_agent)
    workflow.add_node("playwright_test_runner", playwright_test_runner)
    
    workflow.set_entry_point("synthetic_user_story_generator")
    workflow.add_edge("synthetic_user_story_generator", "gherkin_converter")
    workflow.add_edge("gherkin_converter", "code_analyzer")
    workflow.add_edge("code_analyzer", "test_plan_generator")
    workflow.add_edge("test_plan_generator", "playwright_code_generator")
    workflow.add_edge("playwright_code_generator", "code_reviewer")
    workflow.add_conditional_edges(
        "code_reviewer",
        decide_to_refine,
        {
            "refine_code": "code_refiner",
            "set_final_code": "set_final_code"
        }
    )
    workflow.add_edge("code_refiner", "writer_agent")
    workflow.add_edge("set_final_code", "writer_agent")
    workflow.add_edge("writer_agent", "playwright_test_runner")
    workflow.add_edge("playwright_test_runner", END)
    return workflow.compile()



In [17]:
# 8. Test Case Processing
def process_test_case(test_case: dict):
    try:
        case_name = test_case['name']
        logger.info(f"Processing: {case_name}")
        start_time = datetime.now()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Create output directory for this test case
        output_dir = os.path.join(OUTPUT_FOLDER, f"test_case_{case_name}_{timestamp}")
        os.makedirs(output_dir, exist_ok=True)
        
        # Create subdirectories
        for sub in ["features", "tests", "reports"]:
            os.makedirs(os.path.join(output_dir, sub), exist_ok=True)
        
        app = build_graph()
        initial_state = AgentState(
            user_story=test_case["user_story"],
            code_input=test_case["code_input"],
            code_path=test_case["code_path"],
            gherkin_feature="",
            analyzed_code="",
            test_plan="",
            playwright_code="",
            review_feedback="",
            final_playwright_code="",
            test_results=None,
            test_passed=None,
            coverage_report=None,
            coverage_percentage=None,
            output_dir=output_dir,
            artifacts={}
        )
        
        result = app.invoke(initial_state)
        
        duration = (datetime.now() - start_time).total_seconds()
        final_code = result.get("final_playwright_code", "")
        if final_code:
            logger.info(f"{case_name}: Generated executable code ({len(final_code)} chars)")
        else:
            logger.warning(f" {case_name}: No final code generated")
        
        if result.get("test_passed") is not None:
            status = "PASSED" if result["test_passed"] else "FAILED"
            logger.info(f"{case_name}: Test {status}")
        
        coverage = result.get("coverage_percentage")
        if coverage is not None:
            logger.info(f"{case_name}: Coverage {coverage:.1f}%")
        
        logger.info(f" {case_name}: Completed in {duration:.2f}s")
        return case_name, result, output_dir
    except Exception as e:
        logger.error(f" FATAL ERROR in {test_case['name']}: {e}", exc_info=True)
        return test_case["name"], None, None


In [18]:
# 9. Main Execution
if __name__ == "__main__":
    start_time = datetime.now()
    print("_" + "=" * 58 + "_")
    print("UNIVERSAL TEST CONVERTER ")
    print("****" + "=" * 58 + "****")
    logger.info("Starting Universal Test Converter...")
    
    test_cases = load_test_cases("C:\\Projects\\LG\\input_data_1")  # Update path if needed input_data
    if not test_cases:
        logger.error(" No test cases found!")
    else:
        logger.info(f"Found {len(test_cases)} test cases to process")
        results = []
        output_dirs = []
        
        for i, test_case in enumerate(test_cases, 1):
            logger.info(f"\nProcessing {i}/{len(test_cases)}: {test_case['name']}")
            name, result, output_dir = process_test_case(test_case)
            results.append((name, result))
            output_dirs.append(output_dir)
            
            if i < len(test_cases):
                logger.info("Waiting 5 seconds to respect rate limits...")
                time.sleep(5)
        
        # Final Report
        total_time = (datetime.now() - start_time).total_seconds()
        print("\n" + "" + "=" * 58 + "_")
        print("FINAL REPORT ")
        print("_" + "=" * 58 + "_")
        
        passed_count = sum(1 for _, r in results if r and r.get("test_passed") is True)
        failed_count = sum(1 for _, r in results if r and r.get("test_passed") is False)
        incomplete_count = len(results) - passed_count - failed_count
        
        # Calculate average coverage
        coverage_values = [r.get("coverage_percentage") for _, r in results if r and r.get("coverage_percentage") is not None]
        avg_coverage = sum(coverage_values) / len(coverage_values) if coverage_values else 0.0
        
        for name, result in results:
            status = "PASSED" if result and result.get("test_passed") else "FAILED"
            if not result or not result.get("final_playwright_code"):
                status = "INCOMPLETE"
            
            coverage = result.get("coverage_percentage")
            coverage_str = f" (Coverage: {coverage:.1f}%)" if coverage is not None else ""
            print(f"  {name}: {status}{coverage_str}")
        
        print(f"\n STATISTICS:")
        print(f"  • Total Cases: {len(results)}")
        print(f"  • Tests Passed: {passed_count}")
        print(f"  • Tests Failed: {failed_count}")
        print(f"  • Incomplete: {incomplete_count}")
        print(f"  • Success Rate: {(passed_count / len(results) * 100):.1f}%")
        print(f"  • Average Coverage: {avg_coverage:.1f}%")
        print(f"  • Total Time: {total_time:.2f}s")
        print(f"  • Avg Time/Case: {total_time / len(results):.2f}s")
        
        # Create final report
        report_data = {
            "timestamp": datetime.now().isoformat(),
            "total_time": total_time,
            "total_cases": len(results),
            "passed": passed_count,
            "failed": failed_count,
            "incomplete": incomplete_count,
            "success_rate": (passed_count / len(results) * 100),
            "average_coverage": avg_coverage,
            "avg_time_per_case": total_time / len(results),
            "output_dirs": output_dirs
        }
        
        # Save final report
        save_json_artifact(OUTPUT_FOLDER, "final_report.json", report_data)
    
    print("\n Universal Test Converter completed! ")
    print(f"Check the '{OUTPUT_FOLDER}' folder for all generated artifacts.\n")

2025-08-05 08:33:37,353 [INFO] - Starting Universal Test Converter...
2025-08-05 08:33:37,355 [INFO] - Loading test cases from: C:\Projects\LG\input_data_1
2025-08-05 08:33:37,377 [INFO] - Loaded: components
2025-08-05 08:33:37,379 [WARNING] - Incomplete: pages
2025-08-05 08:33:37,380 [WARNING] - Incomplete: styles
2025-08-05 08:33:37,403 [INFO] - Loaded: test_case_1
2025-08-05 08:33:37,424 [INFO] - Loaded: test_case_2
2025-08-05 08:33:37,427 [INFO] -  Total loaded: 3 test cases
2025-08-05 08:33:37,428 [INFO] - Found 3 test cases to process
2025-08-05 08:33:37,429 [INFO] - 
Processing 1/3: components
2025-08-05 08:33:37,430 [INFO] - Processing: components
2025-08-05 08:33:37,458 [INFO] -  Agent: SyntheticUserStoryGenerator - Creating user story from code.


_==========================================================_
UNIVERSAL TEST CONVERTER 
****==========================================================****


2025-08-05 08:33:38,995 [INFO] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 08:33:40,006 [INFO] - Agent: GherkinConverter - Converting user story to Gherkin BDD format.
2025-08-05 08:33:41,221 [INFO] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 08:33:41,226 [INFO] - Agent: CodeAnalyzer - Analyzing input code.
2025-08-05 08:33:43,605 [INFO] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 08:33:45,612 [INFO] - Agent: TestPlanGenerator - Creating comprehensive test plan.
2025-08-05 08:33:47,860 [INFO] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 08:33:49,864 [INFO] - Agent: PlaywrightCodeGenerator - Generating executable test code.
2025-08-05 08:33:51,322 [INFO] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 08:33:53,329 [INFO] - Age


==========================================================_
FINAL REPORT 
_==========================================================_
  components: FAILED (Coverage: 0.0%)
  test_case_1: FAILED (Coverage: 0.0%)
  test_case_2: FAILED (Coverage: 0.0%)

 STATISTICS:
  • Total Cases: 3
  • Tests Passed: 0
  • Tests Failed: 3
  • Incomplete: 0
  • Success Rate: 0.0%
  • Average Coverage: 0.0%
  • Total Time: 95.48s
  • Avg Time/Case: 31.83s

 Universal Test Converter completed! 
Check the 'input_data_output_16' folder for all generated artifacts.

